## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-27-06-46-00 +0000,wsj,Treasury Yields Mostly Rise as Markets Await F...,https://www.wsj.com/finance/investing/short-da...
1,2026-01-27-06-44-51 +0000,nyt,All-Night Concerts in War-Ravaged Myanmar,https://www.nytimes.com/2026/01/27/world/asia/...
2,2026-01-27-06-36-00 +0000,wsj,Comex Gold Futures’ Strong Bullish Momentum St...,https://www.wsj.com/finance/commodities-future...
3,2026-01-27-06-31-09 +0000,bbc,Winter shelter sees highest demand in years,https://www.bbc.com/news/articles/c1lz9218j10o...
4,2026-01-27-06-29-31 +0000,bbc,India and EU set to announce landmark trade deal,https://www.bbc.com/news/articles/crrnee01r9jo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2480/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
61,trump,50
160,minneapolis,23
78,minnesota,22
79,ice,19
172,shooting,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
290,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,132
57,2026-01-27-01-29-20 +0000,nypost,ICE tensions could be thawing in Minn. after T...,https://nypost.com/2026/01/26/us-news/ice-tens...,126
40,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...,122
33,2026-01-27-03-35-02 +0000,nypost,Alex Pretti was among 50 anti-ICE protesters w...,https://nypost.com/2026/01/26/us-news/alex-pre...,113
232,2026-01-26-16-56-15 +0000,latimes,Scrutiny builds over killing of Alex Pretti; T...,https://www.latimes.com/world-nation/story/202...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
290,132,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...
33,113,2026-01-27-03-35-02 +0000,nypost,Alex Pretti was among 50 anti-ICE protesters w...,https://nypost.com/2026/01/26/us-news/alex-pre...
32,66,2026-01-27-03-36-00 +0000,wsj,"Deadly Winter Storm Disrupts Air Travel, Leave...",https://www.wsj.com/us-news/climate-environmen...
40,64,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...
77,55,2026-01-27-00-42-00 +0000,wsj,The Army will gain access to Salesforce’s tech...,https://www.wsj.com/politics/national-security...
330,51,2026-01-26-08-36-39 +0000,wsj,"Stock Market News for Jan. 26, 2026: Dow Rises...",https://www.wsj.com/livecoverage/stock-market-...
76,40,2026-01-27-00-46-17 +0000,nypost,Citi ex-employee accuses top exec of ‘egregiou...,https://nypost.com/2026/01/26/business/citigro...
216,37,2026-01-26-17-54-24 +0000,nypost,At least 18 dead after ferry carrying over 350...,https://nypost.com/2026/01/26/world-news/at-le...
263,37,2026-01-26-14-57-07 +0000,cbc,Israel recovers remains of final hostage in Ga...,https://www.cbc.ca/news/world/gaza-israel-rema...
276,33,2026-01-26-14-01-00 +0000,startribune,"Chris Madel ends GOP bid for governor, says he...",https://www.startribune.com/chris-madel-ends-g...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
